In [1]:
from IPython.display import display, Markdown

## 2. Using k-mers to compare three complete genomes

Let's look at those three genomes from the previous notebook - but now, no longer cut down to 500kb.

To do this, we're going to use the sourmash software, which does all the FracMinHash things but is
much, much faster than using pure Python. (It relies on a programming language called Rust to do
things faster.)

I've pre-calculated k-mer sketches from 3 genomes, one Akkermansia genome (in `2.sig.zip`) and two 
Shewanella genomes (in `47.sig.zip` and `63.sig.zip`) at k=31.

Let's start by asking about the overlap between them:

In [2]:
!sourmash sig overlap inputs/2.sig.zip inputs/47.sig.zip


== This is sourmash version 4.8.11. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

loaded one signature each from inputs/2.sig.zip and inputs/47.sig.zip
first signature:
  signature filename: inputs/2.sig.zip
  signature: CP001071.1 Akkermansia muciniphila ATCC BAA-835, complete genome
  md5: f3a90d4e5528864a5bcc8434b0d0c3b1
  k=31 molecule=DNA num=0 scaled=1000

second signature:
  signature filename: inputs/47.sig.zip
  signature: NC_009665.1 Shewanella baltica OS185, complete genome
  md5: 09a08691ce52952152f0e866a59f6261
  k=31 molecule=DNA num=0 scaled=1000

similarity:                  0.00000
first contained in second:   0.00000
second contained in first:   0.00000

number of hashes in first:   2701
number of hashes in second:  5177

number of hashes in common:  0
only in first:               2701
only in second:              5177
total (union):               7878



In [3]:
!sourmash sig overlap inputs/47.sig.zip inputs/63.sig.zip


== This is sourmash version 4.8.11. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

loaded one signature each from inputs/47.sig.zip and inputs/63.sig.zip
first signature:
  signature filename: inputs/47.sig.zip
  signature: NC_009665.1 Shewanella baltica OS185, complete genome
  md5: 09a08691ce52952152f0e866a59f6261
  k=31 molecule=DNA num=0 scaled=1000

second signature:
  signature filename: inputs/63.sig.zip
  signature: NC_011663.1 Shewanella baltica OS223, complete genome
  md5: 38729c6374925585db28916b82a6f513
  k=31 molecule=DNA num=0 scaled=1000

similarity:                  0.32069
first contained in second:   0.48851
second contained in first:   0.48282

number of hashes in first:   5177
number of hashes in second:  5238

number of hashes in common:  2529
only in first:               2648
only in second:              2709
total (union):               7886



## Displaying relationships with Venn diagrams and upset plots

Let's start by building the Venn diagrams from the previous notebook, but using 
[the betterplot plugin for sourmash instead](https://github.com/sourmash-bio/sourmash_plugin_betterplot/).

In [4]:
!sourmash scripts venn inputs/2.sig.zip inputs/47.sig.zip inputs/63.sig.zip -o venn3.png --ident


== This is sourmash version 4.8.11. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

Loading sketches from inputs/2.sig.zip
...loaded 1 sketches from inputs/2.sig.zip.
Loading sketches from inputs/47.sig.zip
...loaded 1 sketches from inputs/47.sig.zip.
Loading sketches from inputs/63.sig.zip
...loaded 1 sketches from inputs/63.sig.zip.
found three sketches - outputting a 3-part Venn diagram.
saving to 'venn3.png'


In [5]:
display(Markdown("![3-way venn diagram](venn3.png)"))

![3-way venn diagram](venn3.png)

### Venn diagrams vs upset plots

Venn diagrams are good for 2 and 3-way comparisons, but past that they start to get difficult to interpret. An alternative
type of diagram is the [upset plot](https://en.wikipedia.org/wiki/UpSet_plot), which shows relationships between many more sets.

Let's take a look at the same relationships as above, but in an upset plot:

In [6]:
!sourmash scripts upset inputs/2.sig.zip inputs/47.sig.zip inputs/63.sig.zip -o upset.jpg --show-singletons


== This is sourmash version 4.8.11. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

selecting sketches: k=31 scaled=1000 moltype=DNA
loading sketches from file inputs/2.sig.zip
loading sketches from file inputs/47.sig.zip
loading sketches from file inputs/63.sig.zip
Loaded 3 signatures & downsampled to scaled=1000
Showing individual sketch membership b/c of --show-singletons
powerset of distinct combinations: 7
generating intersections...

...done! 4 non-empty intersections of 7 total.
/home/ctbrown/.conda/envs/jup312/lib/python3.12/site-packages/upsetplot/data.py:303: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)
setting min_subset_size='0%' (percentage)
/home/ctbrown/.conda/envs/jup312/lib/python3.12/site-packages/up

In [7]:
display(Markdown("![upset plot](upset.jpg)"))

![upset plot](upset.jpg)

## Calculating some of these numbers in Python

We can also use Python to calculate overlaps/intersections, unions, and various metrics.

The metrics we'll focus on will be:
* Jaccard: intersection of A and B over union of A and B. This tells you how similar A and B are overall.
* containment of B in A: intersection of A and B over size of A. This tells you how much of B is in A.

In [8]:
import sourmash

In [9]:
sig2 = list(sourmash.load_file_as_signatures('inputs/2.sig.zip'))[0]
sketch2 = sig2.minhash

sig47 = list(sourmash.load_file_as_signatures('inputs/47.sig.zip'))[0]
sketch47 = sig47.minhash

sig63 = list(sourmash.load_file_as_signatures('inputs/63.sig.zip'))[0]
sketch63 = sig63.minhash

In [10]:
sketch2.jaccard(sketch47)

0.0

In [11]:
sketch47.jaccard(sketch63)

0.3206949023586102

In [12]:
sketch47.contained_by(sketch63)

0.48850685725323545

In [13]:
sketch63.contained_by(sketch47)

0.48281786941580757

In [14]:
sketch47.contained_by(sketch2)

0.0

Hopefull these numbers are (kind of) clear from the above Venn and upset plots!